In [1]:
import networkx as nx;
#import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;
import numpy as np;

In [2]:
# Max Problem (to obtain the values of x and z given an interdiction)

def InnerProblem (gamma, G, s, t, M):
    
    max_model = gp.Model("Max_z");
    
    x = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    alpha = max_model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    theta = max_model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1);
    
    sum_z = max_model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
            
    max_model.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))
                         - gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    for i,j in G.edges:
        max_model.addConstr(x[i,j] - (1-gamma[i,j].x)*G.edges[i,j]['capacity'] <= 0);
            
    for i,j in G.edges:
        max_model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0);
            
    max_model.addConstr(alpha[t] - alpha[s] >= 1);
            
    max_model.addConstr(x[t,s] - gp.quicksum((1-gamma[i,j].x)*G.edges[i,j]['capacity']*theta[i,j]
                                                     for i,j in G.edges) >= 0);
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            max_model.addConstr(x[i,j] - (1/M)*z[i,j] >= 0);
            
    max_model.addConstr(sum_z <= gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
            
    max_model.setObjective(sum_z, GRB.MAXIMIZE); 
            
    max_model.setParam("IntegralityFocus",1);
    #max_model.setParam("NumericFocus",2);       
    #max_model.setParam('TimeLimit', T_Lim);
            
    max_model.update();
    max_model.setParam("OutputFlag", 0);
    max_model.optimize();
    
    X = {};
    Z = {};
    
    for i,j in G.edges:
        X[i,j] = x[i,j].x;
        Z[i,j] = z[i,j].x;
        
        
      
    return X, Z;


In [3]:
def solver_EarlyRelax (network, budget, rate, T_Limit, summaryName):
    
    start_time = time.time(); 
    networkCSV = network+'.csv';

    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                       cost=data.iat[i,3], special=data.iat[i,4], trafficker=data.iat[i,5], 
                       bottom=data.iat[i,6], victim=data.iat[i,7]);
    
    for i,j in G.edges:           # Applying rate to trafficker's capacity (HT Networks)
        if i == s:
            G.edges[i,j]['capacity'] = math.floor(rate*G.edges[i,j]['capacity']);
            
    A = 0;   # Number of special arcs
    T = 0;   # Number of Traffickers 
    B = 0;   # Number of intermediaries (Bottoms)
    V = 0;   # Number of Victims
    U = 0;   # Maximum capacity
    
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
        if G.edges[i,j]['trafficker'] == 1:
            T = T + 1;
        elif G.edges[i,j]['bottom'] == 1:
            B = B + 1;
        elif G.edges[i,j]['victim'] == 1:
            V = V + 1;
            U = U + G.edges[i,j]['capacity'];
            
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###
    
    
    ### Optimization Model ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    M = A;
    Mo = A;
    Me = A;
    
    instance = 'LogFile_'+network+'_b'+str(budget)+'_r'+str(rate)+'.txt';
    
    model = gp.Model("HT_DelayedRelax");  
    
    gamma = model.addVars(G.edges, vtype=GRB.BINARY);
    chi = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY);
    omega = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    phi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
    eta = model.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
    kappa = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    delta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
    epsilon = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    pi = model.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
    nu = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    psi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    xi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
    
    val = model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    x = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = model.addVars(G.edges, vtype=GRB.BINARY);
    
    
    ### Constraints ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    model.addConstr(val >= gp.quicksum(psi[i,j]*G.edges[i,j]['capacity'] for i,j in G.edges) +
                    pi + gp.quicksum(epsilon[i] for i in G.nodes) + gp.quicksum(nu[i,j] for i,j in G.edges) + 
                    gp.quicksum(kappa[i,j]*G.edges[i,j]['special'] for i,j in G.edges));
    
    model.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget);
            
    model.addConstr(chi[t] - chi[s] + omega[t,s] + eta >= 0);
    
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            model.addConstr(chi[i] - chi[j] + omega[i,j] + phi[i,j] >= 0);
        elif (i != t and j!= s):
            model.addConstr(chi[i] - chi[j] + omega[i,j] >= 0);
            
    for i,j in G.edges:
        if (G.edges[i,j]['special'] == 1):
            model.addConstr(kappa[i,j] - (1/M)*phi[i,j] >= 1);
            
    model.addConstr(gp.quicksum(delta[t,j] for j in G.successors(t))-
                    gp.quicksum(delta[j,t] for j in G.predecessors(t)) + epsilon[t] + pi >= 0);
            
    model.addConstr(gp.quicksum(delta[j,s] for j in G.predecessors(s))-
                    gp.quicksum(delta[s,j] for j in G.successors(s)) + epsilon[s] - pi >= 0);
    
    for i in G.nodes:
        if i != s and i != t:
            model.addConstr(gp.quicksum(delta[i,j] for j in G.successors(i))-
                            gp.quicksum(delta[j,i] for j in G.predecessors(i)) + epsilon[i] >= 0);
                    
    for i,j in G.edges:
        model.addConstr(delta[i,j] - G.edges[i,j]['capacity']*xi[i,j] + nu[i,j] >= 0);
        model.addConstr(psi[i,j] <= omega[i,j]);
        model.addConstr(psi[i,j] <= (1-gamma[i,j])*Mo);
        model.addConstr(psi[i,j] >= omega[i,j] - Mo*gamma[i,j]);
        model.addConstr(xi[i,j] >= eta);
        model.addConstr(xi[i,j] >= -(1-gamma[i,j])*Me);
        model.addConstr(xi[i,j] <= eta + gamma[i,j]*Me);
        
    model.setObjective(val, GRB.MINIMIZE); 
    model.setParam("IntegralityFocus",1);
    #model.setParam("MIPFocus", 3);
    #model.setParam('NodefileStart', 0.5)    # Memore Issues
    #model.setParam("NumericFocus",2);
    model.setParam('TimeLimit', T_Limit); 
    model.update();
    
    model.setParam("LogToConsole", 0)
    model.setParam("OutputFlag", 1);
    model.setParam("LogFile", instance);
    model.optimize();
    
    status = model.status;
    
    ### Solving auxiliary (inner) problem ###
    
    x, z = InnerProblem (gamma, G, s, t, M);
    
    ##########################################
    
    end_time = time.time();
    run_time = round(end_time - start_time, 2);
    
    now = datetime.now();
    
    obj = round(model.objVal);
    LB = model.objBound;
    sol = A - obj;
    OptGap = round(model.MIPGap, 2);
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    
    ### Print to the screen and to result file #######################
    
    file = open('HT_DelayedRelax_'+network+'_b'+str(budget)+'_r'+str(rate)+'.txt', "w");
    file.write('Instance: %s, budget %g, rate %g \n' %(network, budget, rate));
    file.write('Instance executed on:\t %s \n\n' %now.strftime("%c"));
    
    file.write('Number of Nodes: %g' % (T+B+V) +'\n');
    file.write('Number of Traffickers: %g' % T +'\n');
    file.write('Number of Bottoms: %g' % B +'\n');
    file.write('Number of victims: %g' % (V) +'\n');
    file.write('Budget: %g' % budget +'\n\n');
    
    
    print('\n')
    print('Instance: %s, budget %g, rate %g \n' %(network, budget, rate));
    if status == 2:
        print('Status: Optimal');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);
        
        file.write('Status: Optimal\n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);
        
    elif status == 9:
        print('Status: Time Limit');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);     
        print('Optimality Gap: %g' % OptGap);
        
        file.write('Status: Time Limit \n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);     
        file.write('Optimality Gap: %g \n' % OptGap);
              
    else:
        print('Status: %g' % status);
        file.write('Status: %g \n' % status);
    
    print('\nFlow: %g' %x[t,s]);
    file.write('Flow: %g \n' %x[t,s]);
    
    print('run time: %g sec \n' %run_time);
    file.write('run time: %g sec \n' %run_time);
    
    print('Intediction:')
    for i,j in G.edges:
        if gamma[i,j].x > 0.0001:
            print("arc (%g,%g), gamma = %g " %(i,j, gamma[i,j].x));
    print('\n');
    
    file.write('\nTrafficker Capacity: \n');
    for i,j in G.edges:
        if i==s:
            file.write('(%g, %g) : %g \n' %(i,j,G.edges[i,j]['capacity']));
    
    int_T = 0;
    int_B = 0;
    int_V = 0;
    file.write('\nInterdiction plan: \n');
    for i, j in G.edges: 
        if gamma[i,j].x > 0.0001: 
            if G.edges[i,j]['trafficker'] == 1:
                file.write('Trafficker: %g, gamma = %g \n' %(j, gamma[i,j].x));
                int_T = int_T + 1;
            elif G.edges[i,j]['bottom'] == 1:
                file.write('Bottom: %g, gamma = %g \n' %(j, gamma[i,j].x));
                int_B = int_B + 1;
            elif G.edges[i,j]['victim'] == 1:
                file.write('Victim: %g, gamma = %g \n' %(i, gamma[i,j].x));
                int_V = int_V + 1;
    
    file.write('\nVictim nodes with flow \n');    
    for i, j in G.edges:
        if G.edges[i,j]['special'] == 1:
            if x[i,j] > 0.0001:
                file.write('victim: %g, \t Flow: %g \n' %(i, x[i,j]));
    
    file.write('\n');
    file.close();
    
    rowFields = [network, budget, rate, T+B+V, T, B, V, obj, OptGap, x[t,s], int_T, int_B, int_V, run_time];
    
    with open(summaryName, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile);
        csvwriter.writerow(rowFields);
        csvfile.close();
    
    

    

In [ ]:

n_Networks = 3;
Budget = [0, 4, 8, 12, 16, 20, 24];
Rate = [1, 0.75, 0.5];

T_Limit = 30;

summaryName = "HT_DelayedRelax_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance\t, Budget\t, Rate\t, Nodes\t, Traffick\t, Bottoms\t, Victims\t,');
file_summary.write('Obj_value\t, OptGap\t, Flow\t, Int_Traf\t, Int_Bot\t, Int_Vic\t,  Run_Time (sec)\n');
file_summary.close();

for n in range(1, n_Networks+1):
    network = 'Network'+str(n);
    
    for budget in Budget:
        
        for rate in Rate:
            solver_EarlyRelax(network, budget, rate, T_Limit, summaryName);
            
            


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30
Set parameter IntegralityFocus to value 1


Instance: Network1, budget 0, rate 1 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 36

Flow: 195
run time: 0.06 sec 

Intediction:


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30
Set parameter IntegralityFocus to value 1


Instance: Network1, budget 0, rate 0.75 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 36

Flow: 144
run time: 0.05 sec 

Intediction:


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30
Set parameter IntegralityFocus to value 1


Instance: Network1, budget 0, rate 0.5 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 36

Flow: 96
run time: 0.05 sec 

Intediction:


Set p

Set parameter TimeLimit to value 30
Set parameter IntegralityFocus to value 1


Instance: Network2, budget 0, rate 0.5 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 27

Flow: 79
run time: 0.03 sec 

Intediction:


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30
Set parameter IntegralityFocus to value 1


Instance: Network2, budget 4, rate 1 

Status: Optimal
Number of special arcs without flow: 3
Number of special arcs with flow: 24

Flow: 137
run time: 0.12 sec 

Intediction:
arc (9,10), gamma = 1 


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30
Set parameter IntegralityFocus to value 1


Instance: Network2, budget 4, rate 0.75 

Status: Optimal
Number of special arcs without flow: 3
Number of special arcs with flow: 24

Flow: 118
run time: 0.11 sec 

Intediction:
arc (24,25), gamma = 1 


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30
Set parameter